In [1]:
import pandas as pd
import anthropic
import requests
from dotenv import load_dotenv
import base64

import csv
from io import BytesIO
from PIL import Image

load_dotenv()
client = anthropic.Anthropic()

In [2]:
def load_few_shot_examples(csv_file):
    examples = []
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            examples.append({
                'product_name': row['Product_Name'],
                'product_images': eval(row['Product_Images']),
                'product_description': row['Product_Description']
            })
    return examples


def download_and_encode_image(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        image = Image.open(BytesIO(response.content))
        
        # Determine the correct media type
        if image.format.lower() == 'jpeg':
            media_type = 'image/jpeg'
        elif image.format.lower() == 'png':
            media_type = 'image/png'
        elif image.format.lower() == 'gif':
            media_type = 'image/gif'
        elif image.format.lower() == 'webp':
            media_type = 'image/webp'
        else:
            print(f"Unsupported image format: {image.format}")
            return None, None

        buffered = BytesIO()
        image.save(buffered, format=image.format)
        encoded_image = base64.b64encode(buffered.getvalue()).decode('utf-8')
        return encoded_image, media_type
    except Exception as e:
        print(f"Error downloading or encoding image: {e}")
        return None, None
    


## Cache Control the entire user message
BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.0.cache_control: Extra inputs are not permitted'}}

In [3]:
def load_few_shot_pd(few_shot_examples):
    example_messages = []

    for example in few_shot_examples:
        # Prepare user message with product name and images
        user_content = [
            {
                "type": "text", 
                "text": f"Product Name: {example['product_name']}\nProduct Images:"
                }
            ]

        # Download and encode up to 5 valid images
        encoded_images = []
        for url in example['product_images']:
            if len(encoded_images) >= 5:
                break
            encoded_image, media_type = download_and_encode_image(url)
            if encoded_image and media_type:
                encoded_images.append((encoded_image, media_type))

        # Add encoded images to user content
        user_content.extend([
            {
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": media_type,
                    "data": image_data
                }
            } for image_data, media_type in encoded_images
        ])

        # Add user message to example_messages
        example_messages.append({
            "role": "user",
            "content": user_content,
            "cache_control": {"type": "ephemeral"}
        })

        # Add assistant message (product description) to example_messages
        example_messages.append({
            "role": "assistant",
            "content": example['product_description']
        })

    return example_messages

In [4]:
few_shot_examples = load_few_shot_examples('examples/examples_sarcastic.csv')
example_messages = load_few_shot_pd(few_shot_examples[:3]) # load only 3 products to save money

In [5]:
def process_row(row):
    # Prepare user message with product name and images
    user_content = [{"type": "text", "text": f"Product Name: {row['product_name']}\nProduct Images:"}]

    # Download and encode images
    encoded_images = []
    for url in row['product_images']:
        if len(encoded_images) >= 5:
            break
        encoded_image, media_type = download_and_encode_image(url)
        if encoded_image and media_type:
            encoded_images.append((encoded_image, media_type))

    # Add encoded images to user content
    user_content.extend([
        {
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": media_type,
                "data": image_data
            }
        } for image_data, media_type in encoded_images
    ])

    # Prepare the messages for the API call
    messages = example_messages + [
        {"role": "user", "content": user_content}
    ]

    # Make the API call
    response = client.messages.create(
        model="claude-3-5-sonnet-20240620", # claude-3-5-sonnet-20240620
        max_tokens=1000,
        messages=messages,
        extra_headers={"anthropic-beta": "prompt-caching-2024-07-31"}
    )

    # Extract the generated description
    generated_description = response.content[0].text
    
    input_tokens = response.usage.input_tokens
    output_tokens = response.usage.output_tokens

    return generated_description, input_tokens, output_tokens, response

In [6]:
products = []
with open('demo/batch_1.csv', 'r') as file:
    reader = csv.DictReader(file)
    for row in reader:
        products.append({
            'URL': row['URL'],
            'product_name': row['Product_Name'],
            'product_images': eval(row['Product_Images']),
        })

In [7]:
test = products[0]
process_row(test)

BadRequestError: Error code: 400 - {'type': 'error', 'error': {'type': 'invalid_request_error', 'message': 'messages.0.cache_control: Extra inputs are not permitted'}}